# Setup

In [1]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import fsolve
import hmvec as hm
import matplotlib.pyplot as plt
import seaborn as sns

from my_units import *      # where natural units are defined

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcdefaults()
from matplotlib import font_manager
from matplotlib import rcParams
from matplotlib import rc
from matplotlib import colors
rcParams['mathtext.rm'] = 'Times New Roman' 
rcParams['text.usetex'] = True
rcParams['font.family'] = 'times' #'sans-serif'
font_manager.findfont('serif', rebuild_if_missing=True)
fontsize = 14
rcParams.update({'font.size':fontsize})

# Photon to axion conversion functions

## B field profiles

In [3]:
# See Table 1 of 2309.13104 for the halo properties for the given mass bins
file_names = ['profile_bfld_halo_1e10_h12.txt', 'profile_bfld_halo_1e10_h11.txt', 'profile_bfld_halo_1e11_h10.txt', 
              'profile_bfld_halo_1e11_h4.txt', 'profile_bfld_halo_1e12_h12.txt', 'profile_bfld_halo_1e13_h4.txt', 
              'profile_bfld_halo_1e13_h8.txt']
mass_bins = 10**np.array([9.9, 10.4, 10.9, 11.4, 12, 12.5, 13])

In [4]:
# Radial bins are the same for all of the files
rad_bins = np.genfromtxt('./data/bfield_profiles/'+file_names[0], skip_header=3, max_rows=1)
rad_bins_c = rad_bins[:-1]+(rad_bins[1:]-rad_bins[:-1])/2

Bfiled_grid = np.zeros((len(mass_bins), 66, 23))
logB_interp_list = []

for i, file in enumerate(file_names):
    # Magnetic field profiles in muG
    Bfiled_grid[i] = np.genfromtxt('./data/bfield_profiles/'+file_names[i], skip_header=7).astype(float)
#    logB_interp_list.append(RegularGridInterpolator((np.log10(Bfiled_grid[i][:, 0]), rad_bins_c), np.log10(Bfiled_grid[i][:, 3:]*1E6),
#                                                    bounds_error=False, fill_value=-10 ))
    # Use only every other 8 redshift bins to smooth out the profiles
    logB_interp_list.append(RegularGridInterpolator((np.log10(np.concatenate( (Bfiled_grid[i][::8, 0], Bfiled_grid[i][-1:, 0]) )), rad_bins_c),
                                                     np.log10(np.concatenate( (Bfiled_grid[i][::8, 3:], Bfiled_grid[i][-1:, 3:]) )*1E6),
                                                     bounds_error=False, fill_value=-10 )) 

In [5]:
def get_B_rcross(rs, zs, ms, m200c, r200c, rhocritz):
    '''Get the value of the magnetic field at radiai rs'''
    rcross_ratio = rs/r200c
    Brcross = np.zeros((len(zs), len(ms)))

    ms_ind = np.digitize(m200c[0, :], mass_bins)
    ms_ind[ms_ind == len(logB_interp_list)] = len(logB_interp_list)-1.

    for i_m in range(len(ms)):
        for i_z, z_val in enumerate(zs):        
            if rcross_ratio[i_z, i_m] < rad_bins[0]:
                Brcross[i_z, i_m] = ( get_pth_profile(rcross_ratio[i_z, i_m]*r200c[i_z, i_m], z_val, m200c[i_z, i_m], r200c[i_z, i_m], rhocritz[i_z]) /
                                      get_pth_profile(rad_bins[0]*r200c[i_z, i_m], z_val, m200c[i_z, i_m], r200c[i_z, i_m], rhocritz[i_z]) ) * 10.**logB_interp_list[ms_ind[i_m]]( [np.log10(z_val), rad_bins[0]] )   
            else:
                Brcross[i_z, i_m] = 10.**logB_interp_list[ms_ind[i_m]]( [np.log10(z_val), rcross_ratio[i_z, i_m] ] )     
    return Brcross

### Some plot to check

In [6]:
# Some redshift bins
# zMin = 0.005; zMax = 1.9; nZs  = 50
# np.linspace(zMin, zMax, nZs)   
zs  = Bfiled_grid[0, ::5, 0]                 

In [7]:
# Plots of the B field profiles as a function of halo mass and redshift (just for checking)

fig, ax = plt.subplots(2, 4, figsize=(23, 12))
color_list = sns.color_palette("rocket", len(zs))

for i_m in range(len(mass_bins)):
    row_i = int(i_m/4); col_i = i_m%4  
    for i_z, z_val in enumerate(zs):

        pts = np.array([np.full(len(rad_bins_c), np.log10(z_val)), rad_bins_c]).T  
        10**logB_interp_list[i_m](pts)
        
        ax[row_i][col_i].plot(rad_bins_c, 10**logB_interp_list[i_m](pts), color=color_list[i_z], linewidth=2, ls='-')
    
    ax[0][col_i].set_ylim(1E-6, 5)
    ax[1][col_i].set_ylim(0.01, 10)

    ax[row_i][col_i].set_yscale('log')
    ax[row_i][col_i].set_xlabel(r'$r/r_{200c}$', fontsize=15); ax[row_i][0].set_ylabel(r'$B\ [\mu G]$', fontsize=15);
    ax[row_i][col_i].set_title('M = '+"{:.2E}".format(mass_bins[i_m]), fontsize=15 )
    ax[row_i][col_i].grid(ls=':', color='darkgray', alpha=1)
        
ax[1, 3].set_visible(False)

## Gas density profiles

In [8]:
def battagliaAGN(m200, zs):
    '''Parameters of the Battaglia profile'''
    rho0  = 4000. * (m200/1e14)**0.29    * (1.+zs)**(-0.66)
    alpha = 0.88  * (m200/1e14)**(-0.03) * (1.+zs)**0.19
    beta  = 3.83  * (m200/1e14)**0.04    * (1.+zs)**(-0.025)
        
    gamma = -0.2
    xc    = 0.5
    return rho0, alpha, beta, gamma, xc

In [9]:
def get_gas_profile(rs, zs, m200, r200, rhocritz, name='battagliaAGN'):
    '''Get the gas density profile at radiai rs'''
    
    if name!='battagliaAGN':
        print('This gas profile is not implemented. Using battagliaAGN instead.')
    rho0, alpha, beta, gamma, xc = battagliaAGN(m200, zs)

    rho = rhocritz * rho0
    x = rs/r200/xc
    expo = -(beta+gamma)/alpha # gamma sign must be opposite from Battaglia/ACT paper; typo
    return rho * (x**gamma) * ((1.+x**alpha)**expo)


def get_deriv_gas_profile(rs, zs, m200, r200, rhocritz, name='battagliaAGN'):
    '''Get the derivative of the gas density profile at radiai rs'''
    
    if name!='battagliaAGN':
        print('This gas profile is not implemented. Using battagliaAGN instead.')
    rho0, alpha, beta, gamma, xc = battagliaAGN(m200, zs)

    rho = rhocritz * rho0
    x = rs/r200/xc
    expo = -(alpha+beta+gamma)/alpha
    
    drhodr = rho * (x**gamma) * (1. + x**alpha)**expo * (gamma - x**alpha * beta) / rs

    if hasattr(rs, "__len__"): drhodr[rs==0.] = 0.
    elif rs==0: drhodr = 0.
    return drhodr

## Conversion probability and optical depth

In [10]:
def get_200critz(zs, ms, cs, rhocritz, deltav):
    '''Get m200 and r200'''
    delta_rhos1 = deltav*rhocritz
    delta_rhos2 = 200.*rhocritz
    m200critz = hm.mdelta_from_mdelta(ms, cs, delta_rhos1, delta_rhos2)
    r200critz = hm.R_from_M(m200critz, rhocritz[:,None], delta=200.)
    return m200critz, r200critz


def get_rcross_per_halo(zs, ms, rs, rvir, rhocritz, deltav, cs, MA, name='battagliaAGN'):
    '''Compute crossing radius of each halo, i.e. radius where plasma mass^2 = dark photon mass^2'''

    m200critz, r200critz = get_200critz(zs, ms, cs, rhocritz, deltav)
    
    conv = (ombh2/omch2)*ElectronCharge**2./(MElectron*MProton)*MSolar/Mpc**3/eV**2

    rcross_res = np.zeros((len(zs), len(ms)))
    for i_z, z in enumerate(zs):
        for i_m, m in enumerate(ms):
            func = lambda x: np.abs(get_gas_profile(x, z, m200critz[i_z, i_m], r200critz[i_z, i_m], rhocritz[i_z], name=name) * conv/MA**2. - 1.)
            rcross_res[i_z, i_m] = fsolve(func, x0=rs[0])
    return rcross_res

In [11]:
ombh2  = 0.02225                 # Physical baryon density parameter Ωb h2
omch2  = 0.1198                  # Physical dark matter density parameter Ωc h2

def conv_prob(zs, ms, rs, rvir, rhocritz, deltav, cs, MA, rcross, name='battagliaAGN'):
    '''Photon-to-axion conversion probability without including the coupling g and the photon frequency
    Number returned in natural units in GeV''' 
    
    m200, r200 = get_200critz(zs, ms, cs, rhocritz, deltav)
    drhodr = get_deriv_gas_profile(rcross, zs[:,None], m200, r200, rhocritz[:,None], name=name)
    
    # derivative of the plasma mass squared at the resonance crossing radius 
    # rho is in units of MSolar/Mpc^3 and r in Mpc, converting to natural units here
    dm2dr   = np.abs(drhodr)*(ombh2/omch2)*ElectronCharge**2./(MElectron*MProton)*MSolar/Mpc**4. 
    
    # magnetic field at the resonance crossing radius 
    # in units of muGauss, converting to natural units here
    bres = get_B_rcross(rcross, zs, ms, m200, r200, rhocritz)*1.E-6*Gauss

    # redshift scaling form the frequency dependence
    omgz   = (1.+zs[:,None])
    
    return np.pi * omgz * bres**2. / dm2dr

def get_Nres(rvir, rcross):
    '''Including only conversion within the virial radius for each halo'''
    return 2.*np.heaviside(rvir - rcross, 0.5)

def get_u00(zs, chis, rcross):
    '''Angular part of the conversion probability'''
    rchis = chis*aa(zs)
    fract = (rcross/rchis[:,None])

    return np.sqrt(np.pi)*fract**2

def get_tau00(Nres, u00, prob):
    return np.sqrt(4*np.pi) / 3. * Nres * u00 * prob

def get_avtau(zs, ms, nzm, dvol, tau00):
    # Average optical depth per redshift bin
    dtaudz = np.trapz(nzm * tau00, ms, axis=-1) * dvol 
    avtau  = np.trapz(dtaudz, zs, axis=0)
    return avtau, dtaudz

# Example calculation of the monopole (sky-average optical depth)

In [ ]:
############# Units ################
# The units from hmvec are:
# proper radius r is always in Mpc
# comoving momentum k is always in Mpc-1
# All masses m are in Msolar
# rho densities are in Msolar/Mpc^3
# No h units anywhere

In [13]:
# choose redshift range:
zMin, zMax = 0.005, 1.9  

# rmin and rmax are the radial limits to scan over in each halo
# need to make sure crossing radius is within these limits
rMin, rMax = 1e-6, 5e1

aa = lambda z: 1./(1.+z)

# how many bins of each: mass, redshift, wavenumber
# to speed things up, you can choose smaller numbers
nMs = 100
nZs = 100
nKs = 1001

# geomspace divides the interval logarithmically and linspace linearly
ms  = np.geomspace(1e11, 1e17, nMs)      # all masses
zs  = np.linspace(zMin, zMax, nZs)       # all redshifts
rs  = np.linspace(rMin, rMax, 100000)    # halo radial coords
ks  = np.geomspace(1e-4, 1e3, nKs)       # all wavenumbers

# Halo Model
hcos = hm.HaloModel(zs, ks, ms=ms, mass_function='tinker', mdef='vir')

In [14]:
# Useful halo model quantities:

chis     = hcos.comoving_radial_distance(zs)
rvirs    = np.asarray([hcos.rvir(ms,zz) for zz in zs])
cs       = hcos.concentration()
Hz       = hcos.h_of_z(zs)
nzm      = hcos.get_nzm()
biases   = hcos.get_bh()
deltav   = hcos.deltav(zs)
rhocritz = hcos.rho_critical_z(zs)
dvols  = chis**2. / Hz

In [15]:
# Select axion mass in eV, coupling in GeV^-1, and photon frequency
MA = 5e-13
gaphoton = 1e-10
omega = 2.*np.pi*145*GHz

In [16]:
# crossing radius in each halo:
rcross = get_rcross_per_halo(zs, ms, rs, rvirs, rhocritz, deltav, cs, MA)

In [17]:
# computing optical depth in each halo (see Eqs. 2.8-2.10 of 2405.08059)
prob = gaphoton**2*omega*conv_prob(zs, ms, rs, rvirs, rhocritz, deltav, cs, MA, rcross)
Nres = get_Nres(rvirs, rcross)
u00 = get_u00(zs, chis, rcross)
tau00 = get_tau00(Nres, u00, prob)

In [18]:
# integrated optical depth (see Eqs. 2.8-2.10 of 2405.08059)
avtau, dtaudz = get_avtau(zs, ms, nzm, dvols, tau00)
avtau

0.000152450449022955

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

ax.plot(zs, dtaudz*zs/(1e-4), ls='-', linewidth=2)

ax.set_ylabel(r'$ d \bar{\tau}/ d \log z$')
ax.set_xlabel(r'$z$')

ax.grid(ls=':', color='darkgray', alpha=1)

plt.show()